In [26]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']
fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [18]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 3 #months

In [19]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [20]:
# index_universe = [index_to_replicate]+etf_under_idx_list

# daily_tr = d.DAY_TO_DAY_TOTAL_RETURN(start = start_date, end = end_date, per = 'M', fill = 'prev', currency = 'USD')
# daily_tr_noNa = f.dropna(daily_tr)
# tr = f.cumprod(daily_tr_noNa + 1)-1

# # req = bql.Request(u.list([index_to_replicate]+etf_under_idx_list),{'TOTAL_RETURN':tr})
# # res = bq.execute(req)
# # bq_res_array_to_pd(res)

# # Get the cumulative performance

# for i in range(len(index_universe)): #We're doing this to get the DF in a way that the columns are the assets and the rows months (in some cases it's difficult to make the monthly alineation, this is why we develop this code)
#     request = bql.Request(index_universe[i], {index_universe[i]: tr}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
#     res = bq.execute(request)
    
#     asset = res[0].df()
#     asset = pd.DataFrame({index_universe[i]: asset[index_universe[i]].tolist(), 'DATE': asset['DATE'].tolist()}, index= list(range(len(asset))))
    
#     year_list = pd.DatetimeIndex(asset['DATE']).year.tolist()
#     month_list = pd.DatetimeIndex(asset['DATE']).month.tolist()
#     date_list = []
#     for j in range(len(year_list)):
#         date_list += [str(year_list[j]) + '-' + str(month_list[j])]
#     asset['aux_date'] = date_list    
    
#     if i == 0:
#         matrix = asset.copy(deep=True)
#     else :
#         asset = asset.drop(['DATE'], axis=1)
#         matrix = pd.merge(matrix, asset, how='left', on=['aux_date', 'aux_date'])
        
# aux_matrix = matrix[index_universe]

# add_zeros_df = pd.DataFrame([[0]*len(index_universe)], columns=index_universe)
# aux_matrix = pd.concat([add_zeros_df, aux_matrix]).reset_index().drop(['index'], axis=1)

In [21]:
# #Optimzation of series without UCITS constraints to see which is the optimal allocation

# A = []
# for i in range(len(aux_matrix)):
#     A += [aux_matrix[etf_under_idx_list].iloc[i].values.tolist()]
# A = np.array(A)    
    
# y = aux_matrix[index_to_replicate].tolist()
# y = np.array(y)

# def f(x):
#     return sum((np.matmul(A, x) - y)**2)

# def constraint_total_weight_ub(x):  #Tendra que ser inferior a '1' para relajar ligeramente el modelo
#     return 1 - np.sum(x)

# guess=np.zeros(len(etf_list))

# bnds = [(0,1) for i in range(len(etf_list))]
# results = minimize(fun = f, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [{'type':'eq','fun' : constraint_total_weight_ub}])

In [22]:
#HRP generation funtions

import scipy.cluster.hierarchy as sch

#The distance Matrix is define as :
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist=((1-corr)/2.)**.5 # distance matrix
    return dist

def getQuasiDiag(link):
    # Sort clustered items by distance
    link=link.astype(int)
    sortIx=pd.Series([link[-1,0],link[-1,1]])
    numItems=link[-1,3] # number of original items
    while sortIx.max()>=numItems:
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems
        sortIx[i]=link[j,0] # item 1
        df0=pd.Series(link[j,1],index=i+1)
        sortIx=sortIx.append(df0) # item 2
        sortIx=sortIx.sort_index() # re-sort
        sortIx.index=range(sortIx.shape[0]) # re-index
    return sortIx.tolist()

#Función para calcular la inversa de la varianza del portfolio:   
def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp

#Función para calcular la varianza por cada cluster:
def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

#Función Recursive Bisection:
def getRecBipart(cov,sortIx):
    # Compute HRP alloc
    w=pd.Series(1,index=sortIx)
    cItems=[sortIx] # initialize all items in one cluster
    while len(cItems)>0:
        cItems=[i[int(j):int(k)] for i in cItems for j,k in ((0,len(i)/2), \
                                                   (len(i)/2,len(i))) if len(i)>1] # bi-section
        for i in range(0,len(cItems),2): # parse in pairs
            cItems0=cItems[i] # cluster 1
            cItems1=cItems[i+1] # cluster 2
            cVar0=getClusterVar(cov,cItems0)
            cVar1=getClusterVar(cov,cItems1)
            alpha=1-cVar0/(cVar0+cVar1)
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2

    return w 

def HierarchicalRiskParity(dataFrame): #Aquí metes las series y te da el reparto de pesos.
    corr = dataFrame.corr()
    dist = correlDist(corr)
    link=sch.linkage(dist,method='single', metric='euclidean')
    #Almacenamos los indices ordenados:
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    cov = dataFrame.cov() #Matriz de covarianzas
    hrp=getRecBipart(cov,sortIx) # Capital allocation
    hrp = pd.DataFrame(data = {'Assets': hrp.index.tolist(),'Weights': hrp.values.tolist()})
    return hrp  

In [23]:
index_universe = etf_under_idx_list

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)
    
    daily_tr = d.DAY_TO_DAY_TOTAL_RETURN(start = aux_date_st, end = aux_date, per = 'M', fill = 'prev', currency = curncy) #we use the series of the past year
    daily_tr_noNa = f.dropna(daily_tr)
    tr = f.cumprod(daily_tr_noNa + 1)-1

    # Get the cumulative performance

    for i in range(len(index_universe)): #We're doing this to get the DF in a way that the columns are the assets and the rows months (in some cases it's difficult to make the monthly alineation, this is why we develop this code)
        request = bql.Request(index_universe[i], {index_universe[i]: tr}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
        res = bq.execute(request)

        asset = res[0].df()
        asset = pd.DataFrame({index_universe[i]: asset[index_universe[i]].tolist(), 'DATE': asset['DATE'].tolist()}, index= list(range(len(asset))))

        year_list = pd.DatetimeIndex(asset['DATE']).year.tolist()
        month_list = pd.DatetimeIndex(asset['DATE']).month.tolist()
        date_list = []
        for j in range(len(year_list)):
            date_list += [str(year_list[j]) + '-' + str(month_list[j])]
        asset['aux_date'] = date_list    

        if i == 0:
            matrix = asset.copy(deep=True)
        else :
            asset = asset.drop(['DATE'], axis=1)
            matrix = pd.merge(matrix, asset, how='left', on=['aux_date', 'aux_date'])

    aux_matrix = matrix[index_universe]

    add_zeros_df = pd.DataFrame([[0]*len(index_universe)], columns=index_universe)
    aux_matrix = pd.concat([add_zeros_df, aux_matrix]).reset_index().drop(['index'], axis=1)
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    port = HierarchicalRiskParity(aux_matrix)
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': port['Assets'].tolist(), 'weight': port['Weights'].tolist()})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': port['Assets'].tolist(), 'weight': port['Weights'].tolist()})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)
    
port_df
# port_df.set_index(['date', 'security'], inplace = True)
# portfolio_obj = bqport.new_portfolio(from_ = port_df, currency = curncy, type_ = PositionType.DRIFTING_WEIGHT, name = 'Replicador AA Opti HRP Teorico')
# portfolio_obj.save()
# print('Exportado a PRTU')    

2006-01-01 00:00:00
2006-04-01 00:00:00
2006-07-01 00:00:00
2006-10-01 00:00:00
2007-01-01 00:00:00
2007-04-01 00:00:00
2007-07-01 00:00:00
2007-10-01 00:00:00
2008-01-01 00:00:00
2008-04-01 00:00:00
2008-07-01 00:00:00
2008-10-01 00:00:00
2009-01-01 00:00:00
2009-04-01 00:00:00
2009-07-01 00:00:00
2009-10-01 00:00:00
2010-01-01 00:00:00
2010-04-01 00:00:00
2010-07-01 00:00:00
2010-10-01 00:00:00
2011-01-01 00:00:00
2011-04-01 00:00:00
2011-07-01 00:00:00
2011-10-01 00:00:00
2012-01-01 00:00:00
2012-04-01 00:00:00
2012-07-01 00:00:00
2012-10-01 00:00:00
2013-01-01 00:00:00
2013-04-01 00:00:00
2013-07-01 00:00:00
2013-10-01 00:00:00
2014-01-01 00:00:00
2014-04-01 00:00:00
2014-07-01 00:00:00
2014-10-01 00:00:00
2015-01-01 00:00:00
2015-04-01 00:00:00
2015-07-01 00:00:00
2015-10-01 00:00:00
2016-01-01 00:00:00
2016-04-01 00:00:00
2016-07-01 00:00:00
2016-10-01 00:00:00
2017-01-01 00:00:00
2017-04-01 00:00:00
2017-07-01 00:00:00
2017-10-01 00:00:00
2018-01-01 00:00:00
2018-04-01 00:00:00


date        security    weight
0  2006-01-01       RTY Index  0.006146
1  2006-01-01  BFU5TRUU Index  0.110591
2  2006-01-01  NDUEACWF Index  0.004913
3  2006-01-01    FTAW01 Index  0.003247
4  2006-01-01    M1WOEV Index  0.001211
5  2006-01-01  MIMUEMRN Index  0.001657
6  2006-01-01  NDDUEAFE Index  0.007895
7  2006-01-01   NDDUE15 Index  0.025389
8  2006-01-01    BCIW1A Index  0.349043
9  2006-01-01  IDCOT3TR Index  0.489908
0  2006-04-01       RTY Index  0.002497
1  2006-04-01  BFU5TRUU Index  0.108357
2  2006-04-01   NDDUE15 Index  0.002107
3  2006-04-01  NDDUEAFE Index  0.000688
4  2006-04-01    M1WOEV Index  0.000638
5  2006-04-01  MIMUEMRN Index  0.000786
6  2006-04-01  NDUEACWF Index  0.004560
7  2006-04-01    FTAW01 Index  0.008630
8  2006-04-01    BCIW1A Index  0.255694
9  2006-04-01  IDCOT3TR Index  0.616043
0  2006-07-01  IDCOT3TR Index  0.845356
1  2006-07-01    BCIW1A Index  0.092539
2  2006-07-01       RTY Index  0.003216
3  2006-07-01  BFU5TRUU Index  0.054783
4  2006-07-01   NDDUE15 Index  0.001365
5  2006-07-01  MIMUEMRN Index  0.000223
6  2006-07-01  NDUEACWF Index  0.001078
7  2006-07-01    FTAW01 Index  0.000993
8  2006-07-01  NDDUEAFE Index  0.000209
9  2006-07-01    M1WOEV Index  0.000237
..        ...             ...       ...
0  2018-07-01  IDCOT3TR Index  0.492237
1  2018-07-01       RTY Index  0.013370
2  2018-07-01  BFU5TRUU Index  0.440530
3  2018-07-01    BCIW1A Index  0.042516
4  2018-07-01  MIMUEMRN Index  0.003445
5  2018-07-01   NDDUE15 Index  0.002962
6  2018-07-01  NDUEACWF Index  0.001655
7  2018-07-01    FTAW01 Index  0.001494
8  2018-07-01  NDDUEAFE Index  0.001198
9  2018-07-01    M1WOEV Index  0.000591
0  2018-10-01  IDCOT3TR Index  0.671390
1  2018-10-01       RTY Index  0.009857
2  2018-10-01  BFU5TRUU Index  0.263346
3  2018-10-01  NDUEACWF Index  0.009057
4  2018-10-01    FTAW01 Index  0.008422
5  2018-10-01    M1WOEV Index  0.004774
6  2018-10-01  NDDUEAFE Index  0.008453
7  2018-10-01   NDDUE15 Index  0.009719
8  2018-10-01  MIMUEMRN Index  0.001217
9  2018-10-01    BCIW1A Index  0.013765
0  2019-01-01       RTY Index  0.003183
1  2019-01-01  MIMUEMRN Index  0.002467
2  2019-01-01    BCIW1A Index  0.021622
3  2019-01-01  NDUEACWF Index  0.002542
4  2019-01-01    FTAW01 Index  0.003559
5  2019-01-01    M1WOEV Index  0.004937
6  2019-01-01  NDDUEAFE Index  0.004804
7  2019-01-01   NDDUE15 Index  0.009578
8  2019-01-01  BFU5TRUU Index  0.527517
9  2019-01-01  IDCOT3TR Index  0.419792

[530 rows x 3 columns]

In [24]:
#Optimization functions
def f_obj(x):
    return sum((np.matmul(A, x) - y)**2)

def constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

In [27]:
#Add to the portfolio df an auxiliar date 
year_list = pd.DatetimeIndex(port_df['date']).year.tolist()
month_list = pd.DatetimeIndex(port_df['date']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
port_df['aux_date'] = date_list    

#Get the whole price series por all the indexes of the ETFs
download_start_date = start_date
download_end_date = end_date + relativedelta(months=rotation)

price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(u.list(list(port_df['security'].unique())), {'Price': price})
res = bq.execute(request)
price_series = res[0].df().reset_index()

#Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

#Add to the price series df an auxiliar date 
year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
align_price_series['aux_date'] = date_list    

#Get the price and consecuently the number of shares for each moment of the rebalancing date
list_reb_dates = port_df['aux_date'].unique().tolist()

price_0_list = []
for i_date in list_reb_dates:
    aux_port_df = port_df.loc[port_df['aux_date'].isin([i_date])]
    price_0_list += align_price_series.loc[align_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
port_df['Price'] = price_0_list
port_df['N_Assets_type'] = port_df['weight']/port_df['Price']

#For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
aux_df_real_n_assets = port_df.loc[port_df['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]
    
for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
    n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
    cur_price = np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['Price'].tolist())
    val = sum(n_assets*cur_price)
    cur_n_assets = val*np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['weight'].tolist())/cur_price
    
    aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

port_df['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type']

#Put the portfolio df with the number of shares in each period of time in the same format as the price series df
port_df_pivot = port_df.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()
n_assets_type_table = pd.merge(align_price_series['aux_date'].to_frame(), port_df_pivot, how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

#Compute the portfolio value
port_val = align_price_series.copy()
for asset in etf_under_idx_list:
    port_val[asset] = port_val[asset] * n_assets_type_table[asset]
    
port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

#Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(index_to_replicate, {'Price': price})
res = bq.execute(request)
index_price_series = res[0].df().reset_index()

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
def_series = index_price_series.copy()
def_series = def_series.drop(['ID', 'CURRENCY'], axis=1)
def_series = def_series.rename(index=str, columns={'Price': index_to_replicate})
year_list = pd.DatetimeIndex(def_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(def_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
def_series['aux_date'] = date_list

aligned_port_vals = []
for i_date in def_series['aux_date'].tolist():
    aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]
    
#Calculate the cumulated total return of both value series
def_series['Port'] = aligned_port_vals
def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

#Sort Columns
def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

#Plot the result
plot_def_series = def_series.drop(['aux_date'], axis=1).set_index('DATE')
result = bqv.ComparisonPlot(plot_def_series)

# Display the plot
result.show()